In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df=pd.read_csv("../input/Mall_Customers.csv")

In [ ]:
df.info()

So there is no null items.So no need to deal with null values

EDA

In [ ]:
import seaborn as sns
gend_count=df['Gender'].value_counts()
gend_count_scr=pd.DataFrame(df.groupby(['Gender'])['Spending Score (1-100)'].mean())
sns.countplot(x='Gender',data=df)
plt.title("Count of Customers on the basis of Gender")

In [ ]:
gend_count_scr=gend_count_scr.reset_index()
gend_count_scr.plot.bar(x='Gender',y='Spending Score (1-100)')
plt.title("Spending Score of Customers on the basis of Gender")

In [ ]:
gend_count_sal=pd.DataFrame(df.groupby(['Gender'])['Annual Income (k$)'].mean())
gend_count_sal=gend_count_sal.reset_index()
gend_count_sal.plot.bar(x='Gender',y='Annual Income (k$)')
plt.title("Salary of the customers based on gender")

In [ ]:
age_count=pd.DataFrame(df['Age'].value_counts())
age_count=age_count.reset_index()
df['Age'].plot.hist(by='Age')
plt.title("Distribution of Age of customers")

In [ ]:
age_count_scr=pd.DataFrame(df.groupby('Age')['Spending Score (1-100)'].mean())
age_count_scr=age_count_scr.reset_index()
plt.figure(figsize=(10,5))
plt.bar(age_count_scr['Age'],age_count_scr['Spending Score (1-100)'])
plt.xlabel("Age")
plt.ylabel("Average Scoring Score")

In [ ]:
age_count_sal=pd.DataFrame(df.groupby('Age')['Annual Income (k$)'].mean())
age_count_sal=age_count_sal.reset_index()
plt.figure(figsize=(10,5))
plt.bar(age_count_sal['Age'],age_count_sal['Annual Income (k$)'])
plt.xlabel("Age")
plt.ylabel("Average Annual Income (k$)")

In [ ]:
fig,axes=plt.subplots(1,2,figsize=(15,10))
colors = { 'Male' : 'blue', 'Female': 'red'}
axes[0].scatter(df['Age'],df['Annual Income (k$)'],c=df['Gender'].apply(lambda x:colors[x]))
axes[0].set_xlabel("Age")
axes[0].set_ylabel("Annual Income (k$)")
axes[1].scatter(df['Age'],df['Spending Score (1-100)'],c=df['Gender'].apply(lambda x:colors[x]))
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Spending Score (1-100)')

From the above scatter plot its clear that gender doesn't help much in categorising the clusters cause is equally distributed based on the gender.But the plot between Age and Spending Score some clusters can be seen not very evident ones but somehow some clusters can be formed.Lets apply clustering algorithms to cluster them.


In [ ]:
plt.scatter(df['Annual Income (k$)'],df['Spending Score (1-100)'])
plt.xlabel("Annual Income")
plt.ylabel("Spending Score")

So from the above graph its quite evident that there are 5 clusters when we plot between annual Income and Spending Score.Now lets use clustering algorithm and find out what result it shows.


In [ ]:
from sklearn.cluster import KMeans
squared_dist=[]
for i in range(1,21):
    kmeans=KMeans(n_clusters=i)
    kmeans.fit(df[['Annual Income (k$)','Spending Score (1-100)']])
    squared_dist.append(kmeans.inertia_)

In [ ]:
plt.plot(list(range(1,21)),squared_dist)
plt.xlabel("no of clusters")
plt.ylabel("sum of Squared distances from the clusters")

So From the elbow method we can find the best one for the no of clusters is 5.From the plot of annual income vs spending score we have found the clusters to be 5.So our assumption was correct.Lets visualize the clusters properly.

In [ ]:
kmeans=KMeans(n_clusters=5)
y_predict=kmeans.fit_predict(df[['Annual Income (k$)','Spending Score (1-100)']])

In [ ]:
X=df[['Annual Income (k$)','Spending Score (1-100)']]
plt.scatter(X.iloc[y_predict==0,0],X.iloc[y_predict==0,1],c='red',label='Cluster 1')
plt.scatter(X.iloc[y_predict==1,0],X.iloc[y_predict==1,1],c='blue',label='Cluster 2')
plt.scatter(X.iloc[y_predict==2,0],X.iloc[y_predict==2,1],c='green',label='Cluster 3')
plt.scatter(X.iloc[y_predict==3,0],X.iloc[y_predict==3,1],c='yellow',label='Cluster 4')
plt.scatter(X.iloc[y_predict==4,0],X.iloc[y_predict==4,1],c='violet',label='Cluster 5')
plt.xlabel("Annual Income")
plt.ylabel("Spending Score")
plt.legend()

There are basically 5 clusters                                                                                                                                                                                                                                                                  
* Cluster 1:Annual income and Spending Score both in mid region                                                                                                                               
* Cluster 2:Annual Income is high as well as the score.
* Cluster 3:Annual Income is low but the score is high.
* Cluster 4:Annual Income is low and the score is also low.
* Cluster 5:Annual Income is high but the score is low.


So inorder to retain the already existing customers,we can use this above information.For the category of people in Cluster 3 the annual income is low and the Spending Score is high so we can alert them regarding the latest low-mid price products by sending them emails,messages and introducing them with latest offers and discounts.For the people in Cluster 2 we can alert them regarding the products in mid-high price.And for other customers alert messages should be sent but not that frequently as for the already satisfied customers.


In [ ]:
plt.scatter(df.iloc[y_predict==0,2],df.iloc[y_predict==0,3],c='red',label='Cluster 1')
plt.scatter(df.iloc[y_predict==1,2],df.iloc[y_predict==1,3],c='blue',label='Cluster 2')
plt.scatter(df.iloc[y_predict==2,2],df.iloc[y_predict==2,3],c='green',label='Cluster 3')
plt.scatter(df.iloc[y_predict==3,2],df.iloc[y_predict==3,3],c='yellow',label='Cluster 4')
plt.scatter(df.iloc[y_predict==4,2],df.iloc[y_predict==4,3],c='violet',label='Cluster 5')
plt.xlabel("Age")
plt.ylabel("Annual Income")
plt.legend()

* Cluster 1:They Belong to all Age group.
* Cluster 2:They belong to 25-42 age.They have high score aswell.So products as per their convenience(considering the age,gender) should be suggested.
* Cluster 3:They belong to the 18-35 age group.Its quite evident.They have high score.Similar recommendations as per their convenience.
* Cluster 4:They belong to all age group(Mostly older ones) and the income as well as the score is low.Higher age people with low income are not so active regarding Shopping.It can be found out from the above data.Where as younger genration with low income are quite active in matters of shopping.
* Cluster 5:They belong to all age group(mostly 33-60).High Income and low Score.


In [ ]:
plt.scatter(df.iloc[y_predict==0,2],df.iloc[y_predict==0,4],c='red',label='Cluster 1')
plt.scatter(df.iloc[y_predict==1,2],df.iloc[y_predict==1,4],c='blue',label='Cluster 2')
plt.scatter(df.iloc[y_predict==2,2],df.iloc[y_predict==2,4],c='green',label='Cluster 3')
plt.scatter(df.iloc[y_predict==3,2],df.iloc[y_predict==3,4],c='yellow',label='Cluster 4')
plt.scatter(df.iloc[y_predict==4,2],df.iloc[y_predict==4,4],c='violet',label='Cluster 5')
plt.xlabel("Age")
plt.ylabel("Spending Score")
plt.legend()

So from the above analysis we have to take care of the age group,annual income as well as gender while recommending the products.As well as take care of when to alert the customers and by what means.By this way we can retain the existing customers as well as turn the not so active customers into regular buyers.


So this my first kernel .Hope you all find it useful.Please share you opinions and feedbacks.